In [27]:
import json
from data_utils import read_jsonl, add_lineno
from prompt_utils import generate_path

In [3]:
dataset=read_jsonl('data/leetcode-py.jsonl')

# Overall Coverage

In [38]:
prompt_template=open('prompt/template_base.txt').read()
system_template=open('prompt/system.txt').read()
system_message=system_template.format(lang='python')

In [39]:
initial_prompts = dict()

for data in dataset:
    func_name=data['func_name']
    desc=data['description']
    code=data['python_solution']
    difficulty=data['difficulty']
    target_lines=data['target_lines']

    prompt=prompt_template.format(lang='python', program=code, description=desc, func_name=func_name)    
    concatenated_messages = f"system: {system_message}\nuser: {prompt}"
    initial_prompts[data['task_num']] = concatenated_messages

In [40]:
print(list(initial_prompts.values())[0])

system: You are a professional who writes python test methods. You always respond with a python function without any natural language descriptions. Especially, Your answer should not include any quotation marks or descriptions outside the function definition.

user: Please write a test method for the function 'findMedianSortedArrays' given the following program under test and function description. Your answer should only contain one test input.

Program under test:
----
import math
import itertools
import bisect
import collections
import string
import heapq
import functools
import sortedcontainers
from typing import List, Dict, Tuple, Iterator

class Solution:
  def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
    n1 = len(nums1)
    n2 = len(nums2)
    if n1 > n2:
      return self.findMedianSortedArrays(nums2, nums1)

    l = 0
    r = n1

    while l <= r:
      partition1 = (l + r) // 2
      partition2 = (n1 + n2 + 1) // 2 - partition1
      maxLeft1 

In [16]:
with open('totalcov.json', 'w') as f:
    json.dump(initial_prompts, f, indent=2)

# Line Coverage

In [41]:
initial_prompts = dict()

prompt_template=open('prompt/template_line.txt').read()
system_template=open('prompt/system.txt').read()
system_message=system_template.format(lang='python')

for data in dataset:
    func_name=data['func_name']
    desc=data['description']
    code=data['python_solution']
    difficulty=data['difficulty']
    code_withlineno=add_lineno(code)

    target_lines=data['target_lines']
    task_num = data['task_num']

    for lineno in target_lines:
        code_lines=code.split('\n')
        target_line=code_lines[lineno-1]
        target_line_withlineno=f'{lineno}: {target_line}'

        code_input=code_withlineno
        line_input=target_line_withlineno

        prompt=prompt_template.format(lang='python', program=code_input, description=desc, func_name=func_name, lineno=line_input)
        initial_prompts[f"{task_num}_{lineno}"] = f"{system_message}\n{prompt}"

In [42]:
print(list(initial_prompts.values())[0])

You are a professional who writes python test methods. You always respond with a python function without any natural language descriptions. Especially, Your answer should not include any quotation marks or descriptions outside the function definition.

Please write a test method for the function 'findMedianSortedArrays' given the following program under test and function description. Your answer should only contain one test input.

Program under test:
----
1. import math
2. import itertools
3. import bisect
4. import collections
5. import string
6. import heapq
7. import functools
8. import sortedcontainers
9. from typing import List, Dict, Tuple, Iterator
10. 
11. class Solution:
12.   def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
13.     n1 = len(nums1)
14.     n2 = len(nums2)
15.     if n1 > n2:
16.       return self.findMedianSortedArrays(nums2, nums1)
17. 
18.     l = 0
19.     r = n1
20. 
21.     while l <= r:
22.       partition1 = (l + r) // 2
2

In [22]:
with open('linecov.json', 'w') as f:
    json.dump(initial_prompts, f, indent=2)

# Branch Coverage

In [43]:
initial_prompts = dict()

prompt_template=open('prompt/template_branch.txt').read()
system_template=open('prompt/system.txt').read()
system_message=system_template.format(lang='python')

for data in dataset:
    func_name=data['func_name']
    desc=data['description']
    code=data['python_solution']
    difficulty=data['difficulty']
    code_withlineno=add_lineno(code)

    target_lines=data['target_lines']
    task_num = data['task_num']

    branches=data['blocks']
    for branch in branches:
        startline=branch['start']
        endline=branch['end']

        code_input=code_withlineno

        split_lines=code_withlineno.split('\n')
        target_lines=split_lines[startline-1:endline]
        target_branch_withlineno='\n'.join(target_lines)
        branch_input="\n'''\n"+target_branch_withlineno+"\n'''"

        prompt=prompt_template.format(lang='python', program=code_input, description=desc, func_name=func_name, branch=branch_input)

        initial_prompts[f"{task_num}_{startline}_{endline}"] = f"{system_message}\n{prompt}"

In [44]:
print(list(initial_prompts.values())[0])

You are a professional who writes python test methods. You always respond with a python function without any natural language descriptions. Especially, Your answer should not include any quotation marks or descriptions outside the function definition.

Please write a test method for the function 'findMedianSortedArrays' given the following program under test and function description. Your answer should only contain one test input.

Program under test:
----
1. import math
2. import itertools
3. import bisect
4. import collections
5. import string
6. import heapq
7. import functools
8. import sortedcontainers
9. from typing import List, Dict, Tuple, Iterator
10. 
11. class Solution:
12.   def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
13.     n1 = len(nums1)
14.     n2 = len(nums2)
15.     if n1 > n2:
16.       return self.findMedianSortedArrays(nums2, nums1)
17. 
18.     l = 0
19.     r = n1
20. 
21.     while l <= r:
22.       partition1 = (l + r) // 2
2

In [25]:
with open('branchcov.json', 'w') as f:
    json.dump(initial_prompts, f, indent=2)

# Path Coverage

In [45]:
initial_prompts = dict()

prompt_template=open('prompt/template_path.txt').read()
system_template=open('prompt/system.txt').read()
system_message=system_template.format(lang='python')
dataset=read_jsonl('data/leetcode-py-instrumented.jsonl')
path_dataset=read_jsonl('data/tgt_paths.jsonl')

for i, data in enumerate(dataset):
    task_num = data['task_num']
    func_name=data['func_name']
    desc=data['description']
    code=data['python_solution']
    difficulty=data['difficulty']
    code_withlineno=add_lineno(code)

    log_paths=path_dataset[i]['sampled_paths']
    condition_paths=path_dataset[i]['sampled_condition_paths']
    generated_path_tests=[]
    for j in range(len(log_paths)):
        log_path=log_paths[j]
        condition_path=condition_paths[j]
        #print(log_path, condition_path)
        path_prompt=generate_path(condition_path)

        prompt=prompt_template.format(func_name=func_name, description=desc, program=code_withlineno, path=path_prompt)

        initial_prompts[f"{task_num}_{j}"] = f"{system_message}\n{prompt}"

In [37]:
print(list(initial_prompts.values())[0])

You are a professional who writes python test methods. You always respond with a python function without any natural language descriptions. Especially, Your answer should not include any quotation marks or descriptions outside the function definition.

Please write a test method for the function 'findMedianSortedArrays' given the following program under test and function description. Your answer should only contain one test input.

Program under test:
----
1. import math
2. import itertools
3. import bisect
4. import collections
5. import string
6. import heapq
7. import functools
8. import sortedcontainers
9. from typing import List, Dict, Tuple, Iterator
10. 
11. class Solution:
12.   def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
13.     n1 = len(nums1)
14.     n2 = len(nums2)
15.     if n1 > n2:
16.       return self.findMedianSortedArrays(nums2, nums1)
17. 
18.     l = 0
19.     r = n1
20. 
21.     while l <= r:
22.       partition1 = (l + r) // 2
2

In [35]:
with open('pathcov.json', 'w') as f:
    json.dump(initial_prompts, f, indent=2)